In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime as dt
import folium
import numpy as np
import matplotlib as mpl
from folium.plugins import MarkerCluster
import json
import fiona 
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.ticker as ticker

In [2]:
df=pd.read_csv('MORA_alerts.csv', encoding="ISO-8859-1")


C:\Users\Meredith.Raymer\Anaconda3\envs\geo-env\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
outline = gpd.read_file('ParkOutline.kml', driver='KML')

In [4]:
alertPoints = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.location_lon, df.location_lat),crs={'init' :'epsg:4326'})
alertPoints=alertPoints.to_crs(epsg=4326)

C:\Users\Meredith.Raymer\Anaconda3\envs\geo-env\lib\site-packages\pyproj\crs\crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [5]:
MORAalerts=gpd.clip(alertPoints,outline)

In [6]:
MORAalerts['Quality']=(MORAalerts['reliability']+MORAalerts['report_rating'])
MORAalerts.head()

,Unnamed: 0,alert_char_crc,alert_uuid,uuid_version,current_flag,elt_run_id,total_occurences,alert_type,sub_type,street,...,location_lon,jam_uuid,start_pub_millis,start_pub_utc_timestamp,pub_millis,pub_utc_timestamp,pub_utc_epoch_week,road_type,geometry,Quality
27,28,41f96d53,f67579b8-bd92-3c3b-bd4c-ccd3c5e7c1c7,1,1,1533940866,7581,ROAD_CLOSED,ROAD_CLOSED_EVENT,SR-123,...,-121.554361,NaN,1488718420995,2017-03-05 12:53:40,1488718420995,2017-03-05 12:53:40,9,NaN,POINT (-121.55436 46.78582),6
79,80,b16f7449,5a7a28f5-9a92-30f3-a73a-e79ddf617e40,1,1,1533940866,7581,ROAD_CLOSED,ROAD_CLOSED_EVENT,SR-410,...,-121.540535,NaN,1488718403189,2017-03-05 12:53:23,1488718403189,2017-03-05 12:53:23,9,NaN,POINT (-121.54054 46.86756),6
80,81,ce160e69,2f7ad2de-06ec-3db0-9967-43e43bafaef5,1,1,1533940866,7581,ROAD_CLOSED,ROAD_CLOSED_EVENT,SR-123,...,-121.554520,NaN,1488718421615,2017-03-05 12:53:41,1488718421615,2017-03-05 12:53:41,9,NaN,POINT (-121.55452 46.78636),6
470,471,65c1c577,1eb1acce-c22e-35d3-9d8e-f48dd00b0d57,1,1,1533940866,7581,ROAD_CLOSED,ROAD_CLOSED_EVENT,SR-123,...,-121.554361,NaN,1488718473733,2017-03-05 12:54:33,1488718473733,2017-03-05 12:54:33,9,NaN,POINT (-121.55436 46.78582),6
732,733,9d73a79d,e0e91c15-3cf9-3ccc-b9a7-d346c76f38bc,1,1,1533942429,4334,ROAD_CLOSED,ROAD_CLOSED_EVENT,SR-410,...,-121.539917,NaN,1494092776001,2017-05-06 17:46:16,1494092776001,2017-05-06 17:46:16,18,NaN,POINT (-121.53992 46.86689),6


In [7]:
MORAalerts.Quality.value_counts()

6     24363
7        42
8        33
9        14
10       14
5        12
12        4
11        3
13        1
15        1
Name: Quality, dtype: int64

In [8]:
MORAalerts['pub_utc_timestamp_date']=pd.to_datetime(df['pub_utc_timestamp'])

In [9]:
MORAalerts['date']=MORAalerts["pub_utc_timestamp_date"].dt.strftime('%m/%d/%y')
MORAalerts['month_year']=MORAalerts["pub_utc_timestamp_date"].dt.strftime('%Y-%m')
MORAalerts['year']=MORAalerts["pub_utc_timestamp_date"].dt.strftime('%Y')
MORAalerts['month']=MORAalerts["pub_utc_timestamp_date"].dt.strftime('%B')
MORAalerts['hour']=MORAalerts["pub_utc_timestamp_date"].dt.strftime('%I%p')



In [10]:
df_Q_grouped=MORAalerts.groupby(['Quality']).count()
df_Q_grouped=df_Q_grouped.reset_index()
df_Q_grouped=df_Q_grouped[['Quality','alert_char_crc']].sort_values(by='Quality')
df_Q_grouped

,Quality,alert_char_crc
0,5,12
1,6,24363
2,7,42
3,8,33
4,9,14
5,10,14
6,11,3
7,12,4
8,13,1
9,15,1


In [11]:
MORAalerts['day_of_week'] = MORAalerts['pub_utc_timestamp_date'].dt.day_name()
MORAalerts

,Unnamed: 0,alert_char_crc,alert_uuid,uuid_version,current_flag,elt_run_id,total_occurences,alert_type,sub_type,street,...,road_type,geometry,Quality,pub_utc_timestamp_date,date,month_year,year,month,hour,day_of_week
27,28,41f96d53,f67579b8-bd92-3c3b-bd4c-ccd3c5e7c1c7,1,1,1533940866,7581,ROAD_CLOSED,ROAD_CLOSED_EVENT,SR-123,...,NaN,POINT (-121.55436 46.78582),6,2017-03-05 12:53:40,03/05/17,2017-03,2017,March,12PM,Sunday
79,80,b16f7449,5a7a28f5-9a92-30f3-a73a-e79ddf617e40,1,1,1533940866,7581,ROAD_CLOSED,ROAD_CLOSED_EVENT,SR-410,...,NaN,POINT (-121.54054 46.86756),6,2017-03-05 12:53:23,03/05/17,2017-03,2017,March,12PM,Sunday
80,81,ce160e69,2f7ad2de-06ec-3db0-9967-43e43bafaef5,1,1,1533940866,7581,ROAD_CLOSED,ROAD_CLOSED_EVENT,SR-123,...,NaN,POINT (-121.55452 46.78636),6,2017-03-05 12:53:41,03/05/17,2017-03,2017,March,12PM,Sunday
470,471,65c1c577,1eb1acce-c22e-35d3-9d8e-f48dd00b0d57,1,1,1533940866,7581,ROAD_CLOSED,ROAD_CLOSED_EVENT,SR-123,...,NaN,POINT (-121.55436 46.78582),6,2017-03-05 12:54:33,03/05/17,2017-03,2017,March,12PM,Sunday
732,733,9d73a79d,e0e91c15-3cf9-3ccc-b9a7-d346c76f38bc,1,1,1533942429,4334,ROAD_CLOSED,ROAD_CLOSED_EVENT,SR-410,...,NaN,POINT (-121.53992 46.86689),6,2017-05-06 17:46:16,05/06/17,2017-05,2017,May,05PM,Saturday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657336,657337,8ca4074d,c88fcaef-b332-4c67-97da-90f057ab6cb1,1,1,1619870476,10,ROAD_CLOSED,ROAD_CLOSED_EVENT,SR-410,...,7.0,POINT (-121.54190 46.88819),6,2021-05-01 12:00:31,05/01/21,2021-05,2021,May,12PM,Saturday
657341,657342,77422b39,f087e1c5-6a4b-49e8-b1bd-94637f1632a6,1,1,1619884876,10,ROAD_CLOSED,ROAD_CLOSED_EVENT,SR-410,...,7.0,POINT (-121.52707 46.87047),6,2021-05-01 16:00:33,05/01/21,2021-05,2021,May,04PM,Saturday
657345,657346,45198610,e2f06500-bf38-4d40-be0d-f4f1c134f968,1,1,1619875276,10,ROAD_CLOSED,ROAD_CLOSED_EVENT,SR-410,...,7.0,POINT (-121.54190 46.88819),6,2021-05-01 13:00:34,05/01/21,2021-05,2021,May,01PM,Saturday
657364,657365,49a95610,3f4f0eb9-f06f-4b35-bd06-7597b10019ec,1,1,1619904076,10,ROAD_CLOSED,ROAD_CLOSED_EVENT,SR-410,...,7.0,POINT (-121.53992 46.86689),6,2021-05-01 21:00:38,05/01/21,2021-05,2021,May,09PM,Saturday


In [12]:
df_day_grouped=MORAalerts.groupby(['day_of_week']).count()
df_day_grouped=df_day_grouped.reset_index()
df_day_grouped=df_day_grouped[['day_of_week','alert_char_crc']].sort_values(by='day_of_week')
df_day_grouped

,day_of_week,alert_char_crc
0,Friday,3463
1,Monday,3414
2,Saturday,3360
3,Sunday,3609
4,Thursday,3380
5,Tuesday,4017
6,Wednesday,3244


In [13]:
bins = [0, 7, 11, 16]
names = ['Low', 'Medium', 'High']

MORAalerts['qRating'] = pd.cut(MORAalerts['Quality'], bins, labels=names)
MORAalerts.qRating.value_counts()

Low       24417
Medium       64
High          6
Name: qRating, dtype: int64

In [64]:
MORAalerts['month_day']=MORAalerts["pub_utc_timestamp_date"].dt.strftime('%m/%d')

In [42]:
def label_season(row):
    if row['pub_utc_timestamp_date'].month in [1,2,3,4,11,12] or (row['pub_utc_timestamp_date'].month==5 and row['pub_utc_timestamp_date'].day<=15):
        return('Winter')
    elif row['pub_utc_timestamp_date'].month in [6] or (row['pub_utc_timestamp_date'].month==5 and row['pub_utc_timestamp_date'].day>15) or (row['pub_utc_timestamp_date'].month==10 and row['pub_utc_timestamp_date'].day<15):
        return('Shoulder')
    return('Peak')

In [43]:
MORAalerts['season']=MORAalerts.apply(lambda row:label_season(row), axis=1)

In [44]:
df_season_grouped=MORAalerts.groupby(['season']).count()
df_season_grouped=df_season_grouped.reset_index()
df_season_grouped=df_season_grouped[['season','alert_char_crc']].sort_values(by='season')
df_season_grouped

,season,alert_char_crc
0,Peak,385
1,Shoulder,536
2,Winter,23566


In [45]:
nonclosureAlerts = MORAalerts[df.alert_type != 'ROAD_CLOSED']

C:\Users\Meredith.Raymer\Anaconda3\envs\geo-env\lib\site-packages\geopandas\geodataframe.py:1299: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super(GeoDataFrame, self).__getitem__(key)


In [46]:
df_season_grouped2=nonclosureAlerts.groupby(['season']).count()
df_season_grouped2=df_season_grouped2.reset_index()
df_season_grouped2=df_season_grouped2[['season','alert_char_crc']].sort_values(by='season')
df_season_grouped2

,season,alert_char_crc
0,Peak,106
1,Shoulder,15
2,Winter,1
